# Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import evaluate_segments
import random
from src.Video import Video
from src.Segment import Segment
from src import search
import cv2

## Parameters

In [109]:
NUM_VIDEOS = 10
GRID_SIZE = 2
BINS = [180, 180]

## Load training set

In [112]:
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS)

Loading / processing dataset...
Done processing!

In [113]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:   {:d}".format( len(training_set)) )
print("Num. segments: {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:      {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )

TRAINING SET:
Num. videos:   10
Num. segments: 972
Duration:      3,053.0 s


## Select random test set

In [114]:
test_n_segments = 100
test_set = []
labels = []

for i in range(test_n_segments):
    
    # Find random video
    video = random.choice(training_set)
    
    # Select random segment and add histogram to test set
    segment = random.choice(video.segments)
    test_set.append(segment.histograms)
    labels.append(segment)

In [115]:
# Print statistics
print("TEST SET:")
print("Num. histograms: {:d}".format( len(test_set) ))

TEST SET:
Num. histograms: 100


<br>

## Run model on test set

In [116]:
%timeit search.find(test_set[0], training_set, cv2.HISTCMP_CORREL)
%timeit search.find(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT)
%timeit search.find(test_set[0], training_set, cv2.HISTCMP_INTERSECT)
%timeit search.find(test_set[0], training_set, cv2.HISTCMP_BHATTACHARYYA)
%timeit search.find(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT)
%timeit search.find(test_set[0], training_set, cv2.HISTCMP_KL_DIV)

1 loop, best of 5: 326 ms per loop
1 loop, best of 5: 314 ms per loop
1 loop, best of 5: 320 ms per loop
1 loop, best of 5: 325 ms per loop
1 loop, best of 5: 314 ms per loop
1 loop, best of 5: 322 ms per loop


In [117]:
results = []

for i, histogram in enumerate(test_set):
    print('\rSearching segment {}/{}'.format(i+1, len(test_set)), end='', flush=True)
    
    results.append(search.find(histogram, training_set, cv2.HISTCMP_INTERSECT))

Searching segment 100/100

## Evaluate performance

In [118]:
evaluate_segments(results, labels)

Segment evaluation:
Correct: 99
Wrong:   1
Total:   100
TPR:     99.0%
